In [116]:
import requests
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_players = pd.read_parquet('df_players.parquet')
df_positions = pd.read_parquet('df_positions.parquet')
df_teams = pd.read_parquet('df_teams.parquet')

In [117]:
# Paramètre à modifier : budget total (en M£, ou multiplie x10 si prix en 0.1M£)
my_budget = 1007

df_players = df_players.reset_index(drop=True)  # Indices alignés

# Pondérations pour chaque critère (à ajuster selon l’importance donnée à chaque indicateur)
w_points_per_cost = 0.2  # positif : favorise joueurs rapportant beaucoup de points par rapport à leur coût
w_ict_index = 0.1
w_minutes_last = 0.1 # positif : favorise joueurs ayant joué récemment
w_total_points_last = 0.2 # positif : favorise joueurs en forme
w_ict_last = 0.2   # positif : favorise joueurs en forme
w_xGI_last = 0.1  # Poids ajouté sur xG_recent pour MID et FWD
w_goals_conceded_per_90 = -0.2 # négatif : pénalise DEF et GKP concédant beaucoup de buts
w_saves_per_90 = 0.1 # Poids ajouté sur saves pour DEF et GKP
w_defensive_contribution_per_90 = 0.1 # Poids ajouté sur defensive_contribution_per_90 pour DEF et GKP
w_fdr = -0.4   # négatif : pénalise calendrier difficile
w_total_points_last_per_xGI_last = -0.1 # négatif : met en avant les joueurs sous-côtés

# Variable d’état pour chaque joueur
player_vars = [LpVariable(f"player_{i}", cat=LpBinary) for i in range(len(df_players))]

# Fonction objectif : score multi-indicateurs pondéré
prob = LpProblem("FPL_Optimizer", LpMaximize)
prob += lpSum([
    w_points_per_cost  * df_players.loc[i, 'points_per_cost'] * player_vars[i]
  + w_ict_index       * df_players.loc[i, 'ict_index'] * player_vars[i]
  + w_minutes_last * df_players.loc[i, 'minutes_last'] * player_vars[i]
  + w_total_points_last * df_players.loc[i, 'total_points_last'] * player_vars[i]
  + w_ict_last       * df_players.loc[i, 'ict_last'] * player_vars[i]
  + (w_goals_conceded_per_90 if df_players.iloc[i]['position'] in ['GKP', 'DEF'] else 0) * df_players.iloc[i]['goals_conceded_per_90'] * player_vars[i]
  + (w_xGI_last if df_players.iloc[i]['position'] in ['MID', 'FWD'] else 0) * df_players.iloc[i]['xGI_last'] * player_vars[i]
  + (w_saves_per_90 if df_players.iloc[i]['position'] in ['GKP'] else 0) * df_players.iloc[i]['saves_per_90'] * player_vars[i]
  + (w_defensive_contribution_per_90 if df_players.iloc[i]['position'] in ['GKP', 'DEF'] else 0) * df_players.iloc[i]['defensive_contribution_per_90'] * player_vars[i]
  + w_fdr             * df_players.loc[i, 'fdr_next_6'] * player_vars[i]
  + (w_total_points_last_per_xGI_last if df_players.iloc[i]['position'] in ['MID', 'FWD'] else 0) * df_players.loc[i, 'total_points_last_per_xGI_last'] * player_vars[i]
    for i in range(len(df_players))
    if player_vars[i] == 1
])

# Contraintes d’effectif
prob += lpSum(player_vars) == 15
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'GKP']) == 2
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'DEF']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'MID']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'FWD']) == 3

# Contrainte de budget
prob += lpSum([df_players.loc[i, 'price'] * player_vars[i] for i in range(len(df_players))]) <= my_budget

# Contrainte max 3 joueurs de la même équipe
for club in df_players['team_name'].unique():
    prob += lpSum([
        player_vars[i] for i in range(len(df_players))
        if df_players.loc[i, 'team_name'] == club
    ]) <= 3

# Résolution
prob.solve()

# Résultats : joueurs sélectionnés
selected_players = [df_players.loc[i] for i in range(len(df_players)) if player_vars[i].varValue == 1]
selected_df = pd.DataFrame(selected_players)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.12.1/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/8d2dc9d31cc841a3b5a09774ab7f71e4-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/8d2dc9d31cc841a3b5a09774ab7f71e4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 31 COLUMNS
At line 5219 RHS
At line 5246 BOUNDS
At line 5988 ENDATA
Problem MODEL has 26 rows, 741 columns and 2964 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 190.662 - 0.00 seconds
Cgl0004I processed model has 26 rows, 623 columns (623 integer (553 of which binary)) and 2492 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.909091
Cbc0038I Pass   1: suminf.    0.10101 (2) obj. -190.008 iterations 5
Cbc0038I Pass   2: suminf.    0.00000 (0) obj. -174.075 iterations 1
Cbc0038I Solution f

In [118]:
selected_df.sort_values(by = ['position'])

,first_name,web_name,team_short,team_name,position,price,total_points,points_per_cost,points_per_minutes,fdr_next_6,ict_index,selected_by_percent,selected_rank,form,minutes,transfers_in,transfers_in_event,transfers_out,transfers_out_event,assists,goals_scored,goal_involvements,expected_goals,expected_assists,expected_goal_involvements,GI_on_xGI,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90,minutes_last,xGI_last,xG_last,ict_last,total_points_last,total_points_last_per_xGI_last
0,Marc,Guéhi,CRY,Crystal Palace,DEF,47.0,43.0,0.914894,0.079630,3.333333,18.0,26.0,10,8.8,540.0,1960065,59557,411426,1634,2.0,1.0,3.0,0.39,0.09,0.48,6.250000,0.06,0.00,0.02,0.08,1.14,0.50,7.83,90.000000,0.080000,0.065000,3.000000,7.166667,89.583333
1,Omar,Alderete,SUN,Sunderland,DEF,40.0,36.0,0.900000,0.073922,3.000000,19.8,2.6,112,7.8,487.0,244821,14742,68150,756,1.0,1.0,2.0,0.58,0.33,0.91,2.197802,0.11,0.00,0.06,0.17,1.21,0.74,10.90,81.166667,0.151667,0.096667,3.300000,6.000000,39.560440
2,Marcos,Senesi,BOU,Bournemouth,DEF,48.0,43.0,0.895833,0.079630,3.166667,20.2,18.6,19,8.0,540.0,2182063,59852,174233,1477,2.0,0.0,2.0,0.14,0.37,0.51,3.921569,0.02,0.00,0.06,0.08,0.91,1.17,13.50,90.000000,0.085000,0.023333,3.366667,7.166667,84.313725
6,Tyrick,Mitchell,CRY,Crystal Palace,DEF,50.0,34.0,0.680000,0.062963,3.333333,27.9,2.0,130,6.0,540.0,144282,6574,75765,683,0.0,1.0,1.0,0.28,0.46,0.74,1.351351,0.05,0.00,0.08,0.13,1.14,0.50,8.50,90.000000,0.123333,0.046667,4.650000,5.666667,45.945946
7,Trevoh,Chalobah,CHE,Chelsea,DEF,52.0,35.0,0.673077,0.072917,2.833333,29.1,10.7,41,5.2,480.0,1259381,6518,288935,38595,1.0,2.0,3.0,0.75,0.55,1.30,2.307692,0.14,0.00,0.10,0.24,1.01,1.50,9.75,80.000000,0.216667,0.125000,4.850000,5.833333,26.923077
44,João Pedro,João Pedro,CHE,Chelsea,FWD,78.0,37.0,0.474359,0.072549,2.833333,36.3,67.7,1,5.0,510.0,2666790,12330,1079199,56755,3.0,2.0,5.0,1.50,0.23,1.73,2.890173,0.26,0.00,0.04,0.30,1.27,1.24,4.94,85.000000,0.288333,0.250000,6.050000,6.166667,21.387283
65,Erling,Haaland,MCI,Man City,FWD,143.0,59.0,0.412587,0.117296,3.000000,52.2,50.4,3,11.0,503.0,3351246,132987,613297,911,1.0,8.0,9.0,7.35,0.57,7.92,1.136364,1.32,0.00,0.10,1.42,1.14,0.89,4.29,83.833333,1.320000,1.225000,8.700000,9.833333,7.449495
161,Jean-Philippe,Mateta,CRY,Crystal Palace,FWD,76.0,18.0,0.236842,0.033898,3.333333,26.3,9.8,43,3.8,531.0,994260,12149,901960,11043,0.0,2.0,2.0,3.59,0.44,4.03,0.496278,0.61,0.00,0.07,0.68,1.11,0.51,5.76,88.500000,0.671667,0.598333,4.383333,3.000000,4.466501
3,Robin,Roefs,SUN,Sunderland,GKP,45.0,37.0,0.822222,0.068519,3.000000,13.0,2.7,108,7.0,540.0,315483,8169,88918,954,0.0,0.0,0.0,0.00,0.01,0.01,0.000000,0.00,3.33,0.00,0.00,1.15,0.67,0.00,90.000000,0.001667,0.000000,2.166667,6.166667,3700.000000
13,Guglielmo,Vicario,TOT,Spurs,GKP,51.0,32.0,0.627451,0.059259,2.666667,15.0,12.4,34,3.5,540.0,1083786,6433,326878,4554,0.0,0.0,0.0,0.00,0.01,0.01,0.000000,0.00,3.50,0.00,0.00,1.17,0.67,0.00,90.000000,0.001667,0.000000,2.500000,5.333333,3200.000000


In [119]:
selected_df['price'].sum()

np.float64(1001.0)

In [120]:
selected_df['selected_by_percent'].sum()

np.float64(307.3)